In [1]:
import tensorflow as tf
import numpy as np
#import sys
#sys.path.append(r'.\Pycharm')
#from Captcha import gen_captcha,numbers
from captcha.image import ImageCaptcha #audio语音验证码
from PIL import Image
import random

numbers = ['0','1','2','3','4','5','6','7','8','9']
def gen_random_captcha_text(number=4):
    captcha_text = []
    for i in range(number):
        captcha_text.append(random.choice(numbers))
    return captcha_text

def gen_captcha(number=4):
    img = ImageCaptcha()
    captcha_text = gen_random_captcha_text(number)
    captcha_os= img.generate(captcha_text)#captcha_os为一个jpg的输出流
    captcha_image  = Image.open(captcha_os)#PIL中的open函数允许把输出流转换为jpg
    captcha_data  = np.array(captcha_image)#np.array允许把输出流转为像素值矩阵
    return captcha_text,captcha_image,captcha_data

def text2vec(text):#text变为one hot coding
    vector = np.zeros(40)
    for i in range(len(text)):
        idx = i*10+ord(text[i])-48#ACSII转int
        vector[idx] = 1
    return vector

def gen_batch(size=100):#生成一批数据
    batch_x = np.zeros((size,9600))
    batch_y = np.zeros((size,40))
    for i in range(size):
        text,image,data = gen_captcha()
        batch_x[i,:] = np.mean(image,-1).flatten()/255#rgb转为灰度值，再归一化
        batch_y[i,:] = text2vec(text)#text变为one hot coding
    return batch_x,batch_y
    
with tf.name_scope('input') as scope:#name_scope可以让变量有相同的命名，只是限于tf.Variable的变量，可以看做程序里的一个单元
    x = tf.placeholder(tf.float32,[None,60*160],name='x')#N张图片，每张图片像素60*160，放入灰度图
    y_ = tf.placeholder(tf.float32,[None,4*10],name='y_')#one hot coding 6;[0,0,0,0,0,0,1,0,0,0],每个y_ 4个数字，每个数字用1*10向量表示
    prob = tf.placeholder(tf.float32,name='prob')
    
with tf.name_scope('net') as scope:
    x_ = tf.reshape(x,shape=[-1,60,160,1])
    #卷积神经网络1
    with tf.name_scope('conv1') as scope:
        w_c1 = tf.Variable(0.01*tf.random_normal([3,3,1,32]))#*0.01因为不乘的话数字太大，收敛慢
        b_c1 = tf.Variable(0.1*tf.random_normal([32]))
        conv1 = tf.nn.conv2d(x_,w_c1,strides=[1,1,1,1],padding='SAME')
        conv1 = tf.nn.relu(tf.nn.bias_add(conv1,b_c1))
        conv1 = tf.nn.max_pool(conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
        #conv1 = tf.nn.dropout(conv1,prob)
  
    #卷积神经网络2
    with tf.name_scope('conv2') as scope:
        w_c2 = tf.Variable(0.01*tf.random_normal([3,3,32,64]))#*0.01因为不乘的话数字太大，收敛慢
        b_c2 = tf.Variable(0.1*tf.random_normal([64]))
        conv2 = tf.nn.conv2d(conv1,w_c2,strides=[1,1,1,1],padding='SAME')
        conv2 = tf.nn.relu(tf.nn.bias_add(conv2,b_c2))
        conv2 = tf.nn.max_pool(conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
        #conv2 = tf.nn.dropout(conv2,prob)

    #卷积神经网络3
    with tf.name_scope('conv3') as scope:
        w_c3 = tf.Variable(0.01*tf.random_normal([3,3,64,64]))#*0.01因为不乘的话数字太大，收敛慢
        b_c3 = tf.Variable(0.1*tf.random_normal([64]))
        conv3 = tf.nn.conv2d(conv2,w_c3,strides=[1,1,1,1],padding='SAME')
        conv3 = tf.nn.relu(tf.nn.bias_add(conv3,b_c3))
        conv3 = tf.nn.max_pool(conv3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
        #conv3 = tf.nn.dropout(conv3,prob)

    #Dense
    with tf.name_scope('dense') as scope:
        w_d  = tf.Variable(0.01*tf.random_normal([64*8*20,1024]))
        b_d = tf.Variable(0.1*tf.random_normal([1024]))
        dense = tf.reshape(conv3,[-1,w_d.get_shape().as_list()[0]])
        dense = tf.nn.relu(tf.add(tf.matmul(dense,w_d),b_d))
        dense = tf.nn.dropout(dense,prob)
        
    #输出
    with tf.name_scope('output') as scope:
        w = tf.Variable(0.01*tf.random_normal([1024,40]),name = 'w')
        b = tf.Variable(0.1*tf.random_normal([40]),name = 'b')
        y = tf.matmul(dense,w)+b
    
with tf.name_scope('training') as scope:
    #loss = tf.losses.mean_squared_error(y_, y)
    loss = tf.losses.sigmoid_cross_entropy(y_,y)
    tf.summary.scalar('loss',loss)#记录并统计loss
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
    
with tf.name_scope('predict') as scope:
    p = tf.argmax(tf.reshape(y,[-1,4,10]),2) #在第二个维度将one hot coding转为十进制数，投票的方式
    label = tf.argmax(tf.reshape(y_,[-1,4,10]),2)
    pred = tf.equal(p,label)#判断预测和真实值是否相等
    accuracy = tf.reduce_mean(tf.cast(pred,tf.float32))
    tf.summary.scalar('accuracy',accuracy)
 
       
with tf.Session() as sess:
    merged = tf.summary.merge_all()    
    tf.global_variables_initializer().run()
    writer = tf.summary.FileWriter('./log/demo19',sess.graph)
    
    
    step=0
    for i in range(20000):
        #训练
        x_train,y_train = gen_batch(64)
        _,loss_ = sess.run([optimizer,loss],feed_dict={x:x_train,y_:y_train,prob:0.5})
        print(step,loss_)
        #每100批测试一次准确率
        if step%50 == 0:
            x_test,y_test = gen_batch(50)
            summary,acc = sess.run([merged,accuracy],feed_dict={x:x_test,y_:y_test,prob:1})
            writer.add_summary(summary,step)
            print(acc)
        step = step + 1

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


0 0.69194144
0.08
1 0.5379528
2 0.35865402
3 0.40806055
4 0.4277602
5 0.37759262
6 0.33746558
7 0.34553596
8 0.36840674
9 0.36749414
10 0.3586617
11 0.347474
12 0.33293486
13 0.33415282
14 0.34142405
15 0.34802803
16 0.3436108
17 0.34551597
18 0.3309063
19 0.33209315
20 0.33573213
21 0.3366687
22 0.34187165
23 0.33604527
24 0.33544713
25 0.33201942
26 0.335138
27 0.33589202
28 0.3340655
29 0.3381829
30 0.33566102
31 0.33499962
32 0.3330296
33 0.33344403
34 0.32837513
35 0.3325905
36 0.3308446
37 0.32948494
38 0.3331334
39 0.32966813
40 0.33466747
41 0.33304468
42 0.32952997
43 0.33393747
44 0.3281674
45 0.3280971
46 0.33106562
47 0.33055857
48 0.32793218
49 0.32933313
50 0.3283022
0.075
51 0.3298051
52 0.33006978
53 0.33087492
54 0.3333313
55 0.3325144
56 0.3332591
57 0.33205158
58 0.33014423
59 0.3293007
60 0.32946664
61 0.329786
62 0.32900873
63 0.3285009
64 0.32591122
65 0.3301583
66 0.3309754
67 0.32717776
68 0.32710764
69 0.32967687
70 0.33007014
71 0.32826322
72 0.3280315
73 0.32

562 0.32691127
563 0.32738122
564 0.32808584
565 0.3272351
566 0.3270874
567 0.32548156
568 0.32717565
569 0.32775536
570 0.32711607
571 0.3294413
572 0.32709095
573 0.3284414
574 0.32709128
575 0.32699543
576 0.32700962
577 0.32664126
578 0.3263484
579 0.3259737
580 0.3264372
581 0.32779288
582 0.32636237
583 0.32802382
584 0.32362148
585 0.326643
586 0.3260177
587 0.3274377
588 0.3290407
589 0.32939804
590 0.32797736
591 0.32638168
592 0.3280015
593 0.32698745
594 0.32807142
595 0.32768854
596 0.32771006
597 0.32840428
598 0.32465875
599 0.32563418
600 0.32704836
0.085
601 0.32612818
602 0.3270405
603 0.32741386
604 0.3263967
605 0.32817882
606 0.32808614
607 0.32861716
608 0.32694247
609 0.32767734
610 0.3255474
611 0.32680023
612 0.32614452
613 0.32627276
614 0.3262466
615 0.32811302
616 0.3281711
617 0.32751578
618 0.32725114
619 0.32498407
620 0.32703233
621 0.32842517
622 0.3254164
623 0.32679352
624 0.32750517
625 0.3258283
626 0.32748288
627 0.32743573
628 0.3269437
629 0.3273

1110 0.3271161
1111 0.32689786
1112 0.32701558
1113 0.32679307
1114 0.32695174
1115 0.32768577
1116 0.3262683
1117 0.3265018
1118 0.326864
1119 0.32647023
1120 0.32880038
1121 0.32710123
1122 0.3280992
1123 0.32590738
1124 0.32662186
1125 0.32775074
1126 0.3267972
1127 0.32656667
1128 0.3270549
1129 0.32729453
1130 0.32637876
1131 0.3268195
1132 0.32749367
1133 0.3271722
1134 0.32692236
1135 0.32820082
1136 0.327305
1137 0.32558084
1138 0.32697445
1139 0.32730848
1140 0.32620558
1141 0.32704055
1142 0.32757503
1143 0.3278484
1144 0.3269518
1145 0.32846126
1146 0.32896674
1147 0.32570633
1148 0.32596773
1149 0.3268899
1150 0.32734802
0.09
1151 0.3279609
1152 0.3272414
1153 0.32822984
1154 0.32694536
1155 0.3285993
1156 0.32793957
1157 0.32630435
1158 0.3266859
1159 0.32828265
1160 0.3265727
1161 0.32665402
1162 0.32537642
1163 0.32772058
1164 0.32626265
1165 0.32727194
1166 0.32823175
1167 0.32794732
1168 0.32598585
1169 0.32628018
1170 0.3266019
1171 0.3278854
1172 0.32762218
1173 0.32

1630 0.32523555
1631 0.32480007
1632 0.32738173
1633 0.32695884
1634 0.32558176
1635 0.32759425
1636 0.32764396
1637 0.32601872
1638 0.32550117
1639 0.32581562
1640 0.32467794
1641 0.32632726
1642 0.3262763
1643 0.3254706
1644 0.32557374
1645 0.32495844
1646 0.32642335
1647 0.3260745
1648 0.3259603
1649 0.32604232
1650 0.32567537
0.13
1651 0.32530132
1652 0.32613248
1653 0.3270858
1654 0.32496348
1655 0.32479656
1656 0.32540596
1657 0.32581693
1658 0.32617092
1659 0.32608542
1660 0.32569695
1661 0.32641137
1662 0.32553607
1663 0.3257348
1664 0.32509086
1665 0.32502317
1666 0.32601354
1667 0.32633606
1668 0.32557678
1669 0.32409376
1670 0.32584447
1671 0.3258341
1672 0.3251264
1673 0.3255806
1674 0.32473597
1675 0.32500845
1676 0.32618514
1677 0.3260747
1678 0.32623112
1679 0.32497352
1680 0.32450324
1681 0.3247965
1682 0.32573825
1683 0.3245259
1684 0.32563525
1685 0.3260715
1686 0.3248566
1687 0.32501572
1688 0.32465038
1689 0.3254348
1690 0.32485652
1691 0.32549614
1692 0.32566524
16

...

2664 0.21983775
2665 0.21418893
2666 0.21932665
2667 0.22017741
2668 0.22303648
2669 0.22025926
2670 0.22792658
2671 0.2124023
2672 0.2143977
2673 0.21303323
2674 0.20738654
2675 0.21597819
2676 0.21370769
2677 0.21771193
2678 0.21787795
2679 0.219661
2680 0.23266526
2681 0.2133441
2682 0.22576308
2683 0.22213185
2684 0.22658508
2685 0.22947721
2686 0.2244422
2687 0.22500212
2688 0.22555235
2689 0.22021894
2690 0.22692552
2691 0.21840735
2692 0.21684127
2693 0.22430894
2694 0.21455655
2695 0.20425844
2696 0.20584282
2697 0.22978513
2698 0.22656867
2699 0.20708963
2700 0.2171022
0.635
2701 0.22747302
2702 0.21721998
2703 0.22512464
2704 0.21939564
2705 0.216815
2706 0.22245279
2707 0.22150588
2708 0.21453819
2709 0.20879945
2710 0.21883416
2711 0.22864863
2712 0.20845635
2713 0.21624649
2714 0.23892395
2715 0.22556682
2716 0.23572731
2717 0.21205342
2718 0.21647854
2719 0.2161111
2720 0.21516642
2721 0.20912094
2722 0.2116869
2723 0.21840462
2724 0.23218557
2725 0.22669296
2726 0.213468

3179 0.20415826
3180 0.18407236
3181 0.18130267
3182 0.19262335
3183 0.18411657
3184 0.1915957
3185 0.18596587
3186 0.19255011
3187 0.20051202
3188 0.19965854
3189 0.19345418
3190 0.19336961
3191 0.1864061
3192 0.19005232
3193 0.20277253
3194 0.18838146
3195 0.18289211
3196 0.2165447
3197 0.19454722
3198 0.17795254
3199 0.19806623
3200 0.19460014
0.74
3201 0.18650167
3202 0.19921283
3203 0.19312337
3204 0.19680198
3205 0.19602416
3206 0.18496242
3207 0.1975399
3208 0.2030127
3209 0.17941
3210 0.20188722
3211 0.2001398
3212 0.19604646
3213 0.1985015
3214 0.19208547
3215 0.18531808
3216 0.20832261
3217 0.19069606
3218 0.23067823
3219 0.18468484
3220 0.19242096
3221 0.18251981
3222 0.18942179
3223 0.20272693
3224 0.18866831
3225 0.18889204
3226 0.19455484
3227 0.18874276
3228 0.17732635
3229 0.17929667
3230 0.19201061
3231 0.20291252
3232 0.18043312
3233 0.1824935
3234 0.19756582
3235 0.20611867
3236 0.20296936
3237 0.19063531
3238 0.20431495
3239 0.1870698
3240 0.19206852
3241 0.19560215

3695 0.20357093
3696 0.176128
3697 0.17631136
3698 0.17887887
3699 0.19239235
3700 0.181647
0.86
3701 0.18119015
3702 0.1717879
3703 0.1792852
3704 0.18481815
3705 0.17892301
3706 0.17838874
3707 0.18792248
3708 0.17427215
3709 0.17210782
3710 0.17825171
3711 0.15969071
3712 0.1874891
3713 0.178734
3714 0.16905288
3715 0.16275036
3716 0.18356487
3717 0.16245817
3718 0.18116014
3719 0.16461739
3720 0.1772901
3721 0.18539277
3722 0.16133194
3723 0.17194779
3724 0.17067996
3725 0.18276663
3726 0.16394147
3727 0.19653669
3728 0.18030907
3729 0.16577034
3730 0.16697198
3731 0.18671057
3732 0.17723665
3733 0.16705684
3734 0.17704439
3735 0.1705076
3736 0.1677964
3737 0.17016539
3738 0.19135381
3739 0.16267875
3740 0.16127165
3741 0.16639419
3742 0.17814438
3743 0.17564428
3744 0.17640987
3745 0.1726379
3746 0.17907593
3747 0.18886054
3748 0.17085572
3749 0.15804395
3750 0.16708484
0.815
3751 0.17665179
3752 0.1717464
3753 0.18691544
3754 0.17953405
3755 0.17336921
3756 0.1779992
3757 0.18061

4208 0.1720276
4209 0.15399805
4210 0.16190109
4211 0.16274479
4212 0.17619357
4213 0.1646231
4214 0.15508786
4215 0.15769106
4216 0.16232398
4217 0.15970905
4218 0.15305284
4219 0.16638795
4220 0.15625618
4221 0.16849795
4222 0.16954514
4223 0.16070442
4224 0.16020079
4225 0.14440426
4226 0.16535202
4227 0.14342265
4228 0.16510841
4229 0.16204028
4230 0.16058347
4231 0.15665439
4232 0.15598527
4233 0.17299256
4234 0.16906492
4235 0.15169713
4236 0.17505366
4237 0.17043588
4238 0.18274161
4239 0.1537631
4240 0.16398887
4241 0.17643805
4242 0.15141848
4243 0.16029558
4244 0.16000728
4245 0.15993269
4246 0.16816527
4247 0.15858343
4248 0.16667584
4249 0.17795119
4250 0.14739148
0.805
4251 0.17893174
4252 0.16249199
4253 0.14789566
4254 0.17385466
4255 0.17186563
4256 0.15500586
4257 0.17498556
4258 0.15406394
4259 0.15808505
4260 0.15877423
4261 0.16284698
4262 0.15341368
4263 0.15544352
4264 0.15913089
4265 0.16929236
4266 0.14243862
4267 0.17097941
4268 0.14873563
4269 0.1645493
4270 0

4722 0.15175499
4723 0.15285718
4724 0.15196113
4725 0.16145512
4726 0.14782536
4727 0.15215732
4728 0.16159096
4729 0.1440849
4730 0.17559467
4731 0.1492338
4732 0.1527623
4733 0.17733677
4734 0.15007567
4735 0.13752466
4736 0.16427664
4737 0.16325922
4738 0.15740141
4739 0.15244341
4740 0.14588183
4741 0.15055168
4742 0.16475883
4743 0.1654044
4744 0.14122441
4745 0.16750105
4746 0.15935223
4747 0.15452734
4748 0.16444018
4749 0.15245278
4750 0.16406938
0.89
4751 0.13883494
4752 0.14570396
4753 0.15736656
4754 0.14719374
4755 0.16963868
4756 0.16892068
4757 0.16981196
4758 0.15854672
4759 0.14682695
4760 0.15801662
4761 0.15999898
4762 0.15478659
4763 0.13688748
4764 0.15150315
4765 0.14329429
4766 0.1582279
4767 0.14922632
4768 0.14782321
4769 0.14900349
4770 0.16145611
4771 0.14321518
4772 0.16044474
4773 0.15479702
4774 0.16503754
4775 0.15555677
4776 0.15134576
4777 0.1297679
4778 0.14047548
4779 0.15091921
4780 0.15717585
4781 0.16053899
4782 0.1445046
4783 0.15734829
4784 0.149